In [3]:

import os
import pandas as pd
import re
from lxml import etree
from datetime import datetime

ns = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace"}


def readfiles(normrolelist, pathtofiles, type):
    Frame = pd.DataFrame()
    texts =[]
    norms = []
    persons = []
    relatedPersons =[]
    regests = []
    regest_dates = []
    fromdate=[]
    todate=[]
    years = []
    for filename in os.listdir(pathtofiles):
        print("--------------------------")
        print(filename)
        regesttree = etree.parse(pathtofiles+"/"+filename)
        regestroot = regesttree.getroot()
        actualregest = regestroot.find(".//tei:idno", ns).text
        date = createdate(regestroot.find(".//tei:creation/tei:date", ns))
        Nodes = regestroot.findall(".//tei:roleName[@type='"+type+"']", ns)
        for rs in Nodes:
            print("---------")
            print("-Node: %s" %rs.tag)
            print("--Attribute: %s" %rs.attrib)
            text = rs.xpath("normalize-space(.//text())")
            normed = normrole(normrolelist, text)

            if 'corresp' in rs.attrib:
                relatedPerson = rs.attrib['corresp'].replace("#", "")
            elif 'ref' in rs.attrib:
                relatedPerson = "ref"
            else:
                relatedPerson = "None"
            
            personelement = rs.xpath("ancestor::*[@type='person'][1]")
            print("-ParentNode: %s" %personelement[0].tag)
            if personelement is not None:
                print("--Attribute: %s" %personelement[0].attrib)
                personref = personelement[0].attrib['ref'].replace("#","")
            else:
                personref = " "



            texts.append(normed[0])
            norms.append(normed[1])
            persons.append(personref)
            relatedPersons.append(relatedPerson)
            regests.append(actualregest)
            if len(date) == 2:
                persdate = date
                regest_dates.append(persdate[0])
                fromdate.append("")
                todate.append("")
                years.append(persdate[1])
            elif len(date) == 3:
                fromtodate = date
                regest_dates.append("")
                fromdate.append(fromtodate[0])
                todate.append(fromtodate[1])
                years.append(fromtodate[2])
    
    
    
    Frame[type] = texts
    Frame["normed"] = norms
    Frame["Person"] = persons
    Frame["Related Person"] = relatedPersons
    Frame["Regest"] = regests
    Frame["Year"] = years
    Frame["Date"] = regest_dates
    Frame["from"] = fromdate
    Frame["to"] =todate
    
    
    writer = pd.ExcelWriter('../data/output/xlsx/'+type+'Relations.xlsx')
    Frame.to_excel(writer, type+'Relations', index=False)
    writer.save()
    
    Frame.to_csv("../data/output/csv/"+type+"Relations.csv", sep="\t", encoding="utf-8")





def createdate(node):
    print("Datum: %s" %node.attrib)
    if 'when' in node.attrib:
        when = node.attrib['when']
        date = datetime.strptime(when, '%Y%m%d').date()
        year = str(date).split("-")[0]
        return [date,year]
    elif 'from' in node.attrib:
        fr =node.attrib['from']
        to = node.attrib['to']
        fromdate = datetime.strptime(fr, '%Y%m%d').date()
        todate = datetime.strptime(to, '%Y%m%d').date()
        year = str(fromdate).split("-")[0]
        if fromdate == todate:
            return[fromdate,year]
        else:
            return [fromdate, todate,year]

    


def getrolenormlist():
    role_mapping = pd.read_csv("https://docs.google.com/spreadsheets/d/1_ygvqRnDTg6Rx2wp-RNPvZ3cKQFxfDPo3d7SBDiLO5c/export?gid=0&format=csv", dtype=str)\
    .iloc[:,1:3]
    rolesnormlist = role_mapping.set_index("role").to_dict()

    return rolesnormlist['role_norm']


def normrole(rolenormlist, role):
    if role.lower() in rolenormlist:
        return [role, rolenormlist[role.lower()]]
    elif role in rolenormlist:
        return [role, rolenormlist[role]]
    else:
        return [role, " "]
    
normrolelist = getrolenormlist()
readfiles(normrolelist, "../../regests_QGW_PR_Stadt_ab_1400/done", "staff") #Pfad (Durchsuchung), @type of relation


--------------------------
1482.xml
Datum: {'when': '14010119'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'staff', 'corresp': '#pe__wilhelm_herzog_von_oesterreich'}
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#pe__heinrich_egelhaimer_QGW_II_I_1482'}
--------------------------
1483.xml
Datum: {'when': '14010128'}
--------------------------
1484.xml
Datum: {'when': '14010217'}
--------------------------
1485.xml
Datum: {'when': '14010325'}
--------------------------
1486.xml
Datum: {'when': '14010424'}
--------------------------
1487.xml
Datum: {'when': '14010425'}
--------------------------
1488.xml
Datum: {'when': '14010428'}
--------------------------
1489.xml
Datum: {'when': '14010622'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'staff', 'corresp': '#pe__albrecht_iv_herzog_von_oesterreich'}
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': '#

Datum: {'when': '14041213'}
--------------------------
1608.xml
Datum: {'when': '14041215'}
--------------------------
1609.xml
Datum: {'when': '14041216'}
--------------------------
1610.xml
Datum: {'when': '14041219'}
